In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [2]:
prices = pd.DataFrame()

tickers = ['HSBA.UK', 'BARC.UK', 'LLOY.UK', 'RBS.UK', 'BP.UK', 'RIO.UK', 'AAL.UK']

df_list = []

for ticker in tickers:
    df = pd.read_csv(ticker + '.csv')
    df.Date = pd.to_datetime(df.Date)
    df = df.set_index('Date')
    df_list.append(df)

In [3]:
RDSA = pd.read_csv('RDSA.UK.csv', sep = ';')

RDSA.Close = RDSA.Close.apply(lambda x: x.replace(',','.'))
RDSA.Close = RDSA.Close.astype('float64')
RDSA.Date = pd.to_datetime(RDSA.Date, format='%d/%m/%Y')

RDSA = RDSA.set_index('Date')

df_list.append(RDSA)
tickers.append('RDSA.UK')

In [4]:
prices = pd.concat(df_list, axis=1, join='outer')
prices.columns = tickers
prices = prices.sort_index()

In [5]:
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RIO.UK,AAL.UK,RDSA.UK
Date,,,,,,,,
2009-01-07,427.80,144.11,73.682,486.03,409.64,1148.67,1521.30,18.05
2009-01-08,425.98,150.20,76.360,501.90,416.09,1096.69,1431.35,18.41
2009-01-09,418.16,151.91,78.201,526.70,405.93,1033.30,1314.05,17.88
2009-01-12,424.96,156.68,83.674,545.54,398.11,1018.74,1279.65,17.73
2009-01-13,425.79,140.79,79.094,506.86,403.18,985.76,1260.24,18.02


In [6]:
prices.index

DatetimeIndex(['2009-01-07', '2009-01-08', '2009-01-09', '2009-01-12',
               '2009-01-13', '2009-01-14', '2009-01-15', '2009-01-16',
               '2009-01-19', '2009-01-20',
               ...
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-28',
               '2011-12-29', '2011-12-30', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06'],
              dtype='datetime64[ns]', name='Date', length=758, freq=None)

In [7]:
prices.dtypes

HSBA.UK    float64
BARC.UK    float64
LLOY.UK    float64
RBS.UK     float64
BP.UK      float64
RIO.UK     float64
AAL.UK     float64
RDSA.UK    float64
dtype: object

Датафрейм собран, все в нужном формате. Идем дальше.

## Пропуски

In [8]:
prices.isna().sum()

HSBA.UK    8
BARC.UK    1
LLOY.UK    1
RBS.UK     0
BP.UK      9
RIO.UK     5
AAL.UK     0
RDSA.UK    0
dtype: int64

Заменим пропуски интерполяцией.

In [9]:
prices = prices.interpolate(method = 'time', axis = 0)

In [10]:
prices.isna().sum()

HSBA.UK    0
BARC.UK    0
LLOY.UK    0
RBS.UK     0
BP.UK      0
RIO.UK     0
AAL.UK     0
RDSA.UK    0
dtype: int64

## Получим доходности

In [11]:
prices = prices.pct_change()

In [12]:
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RIO.UK,AAL.UK,RDSA.UK
Date,,,,,,,,
2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,-0.004254,0.042259,0.036345,0.032652,0.015746,-0.045252,-0.059127,0.019945
2009-01-09,-0.018358,0.011385,0.024109,0.049412,-0.024418,-0.057801,-0.081951,-0.028789
2009-01-12,0.016262,0.031400,0.069986,0.035770,-0.019264,-0.014091,-0.026179,-0.008389
2009-01-13,0.001953,-0.101417,-0.054736,-0.070902,0.012735,-0.032373,-0.015168,0.016356


In [13]:
prices = prices[1:]

In [14]:
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RIO.UK,AAL.UK,RDSA.UK
Date,,,,,,,,
2009-01-08,-0.004254,0.042259,0.036345,0.032652,0.015746,-0.045252,-0.059127,0.019945
2009-01-09,-0.018358,0.011385,0.024109,0.049412,-0.024418,-0.057801,-0.081951,-0.028789
2009-01-12,0.016262,0.031400,0.069986,0.035770,-0.019264,-0.014091,-0.026179,-0.008389
2009-01-13,0.001953,-0.101417,-0.054736,-0.070902,0.012735,-0.032373,-0.015168,0.016356
2009-01-14,-0.080039,-0.143405,-0.118770,-0.183956,-0.051887,-0.112553,-0.095164,-0.051054


Поставим колонки по секторам. Сначала - банковский, затем - нефтегаз, в конце - металлургия.

In [17]:
new_order = ['HSBA.UK', 'BARC.UK', 'LLOY.UK', 'RBS.UK', 'BP.UK', 'RDSA.UK', 'RIO.UK', 'AAL.UK']

prices = prices[new_order]
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-08,-0.004254,0.042259,0.036345,0.032652,0.015746,0.019945,-0.045252,-0.059127
2009-01-09,-0.018358,0.011385,0.024109,0.049412,-0.024418,-0.028789,-0.057801,-0.081951
2009-01-12,0.016262,0.031400,0.069986,0.035770,-0.019264,-0.008389,-0.014091,-0.026179
2009-01-13,0.001953,-0.101417,-0.054736,-0.070902,0.012735,0.016356,-0.032373,-0.015168
2009-01-14,-0.080039,-0.143405,-0.118770,-0.183956,-0.051887,-0.051054,-0.112553,-0.095164


In [18]:
prices.to_csv('stock_prices.csv')